<a href="https://colab.research.google.com/github/madhurlak0810/SEC-edgar/blob/test_inesh/Qualitative_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import mongodb as mdb

mdb.get_mongodb_client()


MongoClient(host=['cluster0-shard-00-02.rrvyc.mongodb.net:27017', 'cluster0-shard-00-00.rrvyc.mongodb.net:27017', 'cluster0-shard-00-01.rrvyc.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=False, authsource='admin', replicaset='atlas-95o7b3-shard-0', tls=True)

In [13]:
mdb.download_cik_ticker_map()

In [14]:
tiker = "GOOGL"
cik = mdb.cik_from_ticker(tiker)
cik

'0001652044'

In [16]:
mdb.download_all_cik_submissions(cik)

In [15]:
mdb.download_submissions_documents(cik,"10-K",5)

form
2024-01-31 (10-K): https://www.sec.gov/Archives/edgar/data/1652044/000165204424000022/goog-20231231.htm
form
form
2022-02-02 (10-K): https://www.sec.gov/Archives/edgar/data/1652044/000165204422000019/goog-20211231.htm


In [31]:
filing_url = 'https://www.sec.gov/Archives/edgar/data/1652044/000165204423000016/goog-20221231.htm'
doc = mdb.get_collection("documents").find({"_id":filing_url }).next()

In [19]:
mdb.parse_document(doc)

form type: 		10-K
cik            0001652044
name        Alphabet Inc.
ticker              GOOGL
exchange           Nasdaq
Name: 4, dtype: object


In [20]:
parsed_doc = mdb.get_collection("parsed_documents").find({"_id":filing_url}).next()

In [22]:
import analyzer as an

restructured_doc=an.restructure_parsed_10k(parsed_doc)

In [23]:
section_text = restructured_doc["business"]["text"]
section_text

'ITEM 1. BUSINESS Overview As our founders Larry and Sergey wrote in the original founders\' letter, "Google is not a conventional company. We do not intend to become one." That unconventional spirit has been a driving force throughout our history, inspiring us to tackle big problems and invest in moonshots, such as our long-term opportunities in artificial intelligence (AI). We continue this work under the leadership of Alphabet and Google CEO Sundar Pichai. Alphabet is a collection of businesses -- the largest of which is Google. We report Google in two segments, Google Services and Google Cloud; we also report all non-Google businesses collectively as Other Bets. Alphabet\'s structure is about helping each of our businesses prosper through strong leaders and independence. Access and technology for everyone The Internet is one of the world\'s most powerful equalizers; it propels ideas, people and businesses large and small. Our mission to organize the world\'s information and make it

In [24]:
len(section_text)

25020

In [26]:
chain_type = "refine"
model = "gpt-4o-mini"
verbose = True

# get summary from openAI model
print(f"business original_len: {len(section_text)} use {model} w/ chain {chain_type}")
summary, cost = an.summarize_section(section_text, model, chain_type, verbose)

business original_len: 25020 use gpt-4o-mini w/ chain refine


c:\Users\madhu\OneDrive\Documents\GitHub\SEC-edgar\analyzer.py:81: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = chain.run(docs)




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"ITEM 1. BUSINESS Overview As our founders Larry and Sergey wrote in the original founders' letter, "Google is not a conventional company. We do not intend to become one." That unconventional spirit has been a driving force throughout our history, inspiring us to tackle big problems and invest in moonshots, such as our long-term opportunities in artificial intelligence (AI). We continue this work under the leadership of Alphabet and Google CEO Sundar Pichai. Alphabet is a collection of businesses -- the largest of which is Google. We report Google in two segments, Google Services and Google Cloud; we also report all non-Google businesses collectively as Other Bets. Alphabet's structure is about helping each of our businesses prosper through strong leaders and independence. Access and technology for everyone The Internet is one of the wor

In [27]:
print(f"BULLET POINTS")
for el in summary:
    print(el)
print(f"cost: {cost} in USD")

BULLET POINTS
Alphabet Inc., under the leadership of CEO Sundar Pichai, continues to embody the unconventional spirit outlined by its founders, Larry and Sergey
The company operates primarily through Google, which is divided into two segments: Google Services and Google Cloud, while non-Google ventures are categorized as Other Bets
Google’s mission remains to organize information and enhance accessibility, with a strong focus on innovation in artificial intelligence (AI) to benefit users and businesses alike.

The company prioritizes user privacy and security, ensuring products are built with responsible data practices
Google Services generates revenue primarily through advertising, while Google Cloud provides advanced solutions for cybersecurity, data analytics, and productivity
Alphabet also invests in Other Bets aimed at solving significant societal challenges.

As of December 31, 2022, Alphabet had 190,234 employees and is committed to supporting protected labor rights, maintaining

In [28]:
an.sections_summary(parsed_doc)

business original_len: 25020 use gpt-3.5-turbo-16k w/ chain refine
business original_len: 25020 summary_len: 748 reduction: 97.01% cost: 0.0204$ duration:4.8s used gpt-3.5-turbo-16k w/ chain refine
risk original_len: 82337 use gpt-3.5-turbo-16k w/ chain refine
risk original_len: 82337 summary_len: 2007 reduction: 97.56% cost: 0.0712$ duration:18.4s used gpt-3.5-turbo-16k w/ chain refine
property original_len: 328 use gpt-3.5-turbo w/ chain refine
property original_len: 328 summary_len: 226 reduction: 31.099999999999994% cost: 0.0002$ duration:0.9s used gpt-3.5-turbo w/ chain refine
legal original_len: 272 use gpt-3.5-turbo w/ chain refine
legal original_len: 272 summary_len: 96 reduction: 64.71000000000001% cost: 0.0002$ duration:0.9s used gpt-3.5-turbo w/ chain refine
other original_len: 493 use gpt-3.5-turbo w/ chain refine
other original_len: 493 summary_len: 314 reduction: 36.31% cost: 0.0004$ duration:1.0s used gpt-3.5-turbo w/ chain refine

Total Cost: 0.09240000000000001$, Total

In [29]:
sections=mdb.get_collection("items_summary").find({"_id":filing_url}).next()

In [30]:
sections

{'_id': 'https://www.sec.gov/Archives/edgar/data/1652044/000165204423000016/goog-20221231.htm',
 'name': 'Alphabet Inc.',
 'ticker': 'GOOGL',
 'form_type': '10-K',
 'filing_date': None,
 'business': {'summary': ['Alphabet Inc., the parent company of Google, is committed to supporting protected labor rights and maintaining an open culture',
   'The company communicates information through multiple internal channels and contracts with businesses and staffing agencies when necessary',
   'Alphabet faces scrutiny from governments regarding compliance with laws and regulations, particularly in areas such as data privacy, content moderation, competition, consumer protection, climate change, and diversity',
   'The company relies on intellectual property laws and procedures to protect its technology and brand',
   'Information about Alphabet is available on its website and investor relations website, including financial reports, webcasts, news, and corporate governance information.'],
  'link